In [1]:
%load_ext sql
%matplotlib notebook

In [2]:
%sql postgresql://localhost:6000/mnist

u'Connected: @mnist'

In [3]:
%sql select madlib.version();

 * postgresql://localhost:6000/mnist
1 rows affected.


version
"MADlib version: 1.18.0-dev, git revision: rc/1.17.0-rc2-100-g4987e8fe, cmake configuration time: Fri Mar 12 20:51:57 UTC 2021, build type: debug, build system: Darwin-20.3.0, C compiler: Clang, C++ compiler: Clang"


In [4]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Reshape

# Multi Output Mnist Example

## Define Model

In [ ]:
# creating model
inputs = Input(shape = (28,28))
flatten1 = Flatten()(inputs)
dense1 = Dense(512, activation = 'relu')(flatten1)
dense2 = Dense(128, activation = 'relu')(dense1)
dense3 = Dense(32, activation = 'relu')(dense2)

# create classification output
classification_output = Dense(10, activation = 'softmax')(dense3)

# use output from dense layer 3 to create autoencder output
up_dense1 = Dense(128, activation = 'relu')(dense3)
up_dense2 = Dense(512, activation = 'relu')(up_dense1)
decoded_flat = Dense(784)(up_dense2)
decoded_outputs = Reshape((28,28))(decoded_flat)

In [ ]:
model = Model(inputs, [classification_output,decoded_outputs])
model.summary()

In [ ]:
## From https://theailearner.com/2019/01/25/multi-input-and-multi-output-models-in-keras/
# m = 256
# n_epoch = 25
# model.compile(optimizer='adam', loss=['categorical_crossentropy', 'mse'], loss_weights = [1.0, 0.5], metrics = ['accuracy'])
# model.fit(output_X_train,[Y_train, output_X_train], epochs=n_epoch, batch_size=m, shuffle=True)

## Load Model

In [ ]:
json_model = model.to_json()

In [ ]:
%%sql
DROP TABLE IF EXISTS model_arch_library;
SELECT madlib.load_keras_model('model_arch_library', :json_model ::json,
                               NULL,                  -- Weights
                               'Multi Output MNIST',              -- Name
                               'Output 1: classification, Output 2: autoencoder' -- Descr
);

SELECT * FROM model_arch_library;

In [ ]:
%sql \dt

In [ ]:
%sql SELECT * FROM model_arch_library

## Run Preprocessor

In [ ]:
%%sql
DROP TABLE IF EXISTS mnist_train_data_packed, mnist_train_data_packed_summary;
SELECT madlib.training_preprocessor_dl('mnist_train_data',        -- Source table
                                       'mnist_train_data_packed', -- Output table
                                       'y',                          -- Dependent variable
                                       'x',                          -- Independent variable
                                        1000,                        -- Buffer size
                                        255                          -- Normalizing constant
                                        );

SELECT * FROM mnist_train_data_packed_summary;

In [ ]:
%%sql
DROP TABLE IF EXISTS mnist_test_data_packed, mnist_test_data_packed_summary;

SELECT madlib.training_preprocessor_dl('mnist_test_data',        -- Source table
                                       'mnist_test_data_packed', -- Output table
                                       'y',                          -- Dependent variable
                                       'x',                          -- Independent variable
                                        1000,                        -- Buffer size
                                        255                          -- Normalizing constant
                                        );

SELECT * FROM mnist_train_data_packed_summary;

In [ ]:
%sql SET optimizer=off

In [ ]:
%sql SHOW optimizer

## Fit Model

In [ ]:
%%sql
DROP TABLE IF EXISTS mnist_multout_model, mnist_multout_model_summary;

SELECT madlib.madlib_keras_fit('mnist_train_data_packed',    -- source table
                               'mnist_multout_model',                -- model output table
                               'model_arch_library',            -- model arch table
                                1,                              -- model arch id
                                $$ loss='mse', loss_weights = [1.0, 0.5], optimizer='adam', metrics=['accuracy']$$,  -- compile_params
                                $$ batch_size=256, epochs=1, shuffle=True $$,  -- fit_params
                                3,                              -- num_iterations
                                FALSE,                          -- use GPUs
                                'mnist_test_data_packed',    -- validation dataset
                                2                               -- metrics compute frequency
                              );

## Evaluate

In [ ]:
%%sql
DROP TABLE IF EXISTS mnist_multout_eval, mnist_multout_eval_summary;
SELECT madlib.madlib_keras_evaluate('mnist_model',               -- model
                                    'mnist_test_data_packed',   -- test table
                                    'mnist_multout_eval'             -- output table
                                    );

SELECT * FROM cifar10_validate;

## Predict

In [ ]:
%%sql
DROP TABLE IF EXISTS mnist_multout_predict, mnist_multout_predict;
SELECT madlib.madlib_keras_predict()

## Plots

In [ ]:
# get accuracy and iteration number
iters_proxy = %sql SELECT metrics_iters FROM cifar_10_model_summary;
train_accuracy_proxy = %sql SELECT training_metrics FROM cifar_10_model_summary;
test_accuracy_proxy = %sql SELECT validation_metrics FROM cifar_10_model_summary;

# get number of points
num_points_proxy = %sql SELECT array_length(metrics_iters,1) FROM cifar_10_model_summary;
num_points = num_points_proxy[0]

# reshape to np arrays
iters = np.array(iters_proxy).reshape(num_points)
train_accuracy = np.array(train_accuracy_proxy).reshape(num_points)
test_accuracy = np.array(test_accuracy_proxy).reshape(num_points)

#plot
plt.title('CIFAR-10 accuracy by iteration')
plt.xlabel('Iteration number')
plt.ylabel('Accuracy')
plt.grid(True)
plt.plot(iters, train_accuracy, 'g.-', label='Train')
plt.plot(iters, test_accuracy, 'r.-', label='Test')
plt.legend()

Loss by iteration

# get loss
train_loss_proxy = %sql SELECT training_loss FROM cifar_10_model_summary;
test_loss_proxy = %sql SELECT validation_loss FROM cifar_10_model_summary;

# reshape to np arrays
train_loss = np.array(train_loss_proxy).reshape(num_points)
test_loss = np.array(test_loss_proxy).reshape(num_points)

#plot
plt.title('CIFAR-10 loss by iteration')
plt.xlabel('Iteration number')
plt.ylabel('Loss')
plt.grid(True)
plt.plot(iters, train_loss, 'g.-', label='Train')
plt.plot(iters, test_loss, 'r.-', label='Test')
plt.legend()

Accuracy by time

# get time
time_proxy = %sql SELECT metrics_elapsed_time FROM cifar_10_model_summary;

# reshape to np arrays
time = np.array(time_proxy).reshape(num_points)/60.0

#plot
plt.title('CIFAR-10 accuracy by time')
plt.xlabel('Time (min)')
plt.ylabel('Accuracy')
plt.grid(True)
plt.plot(time, train_accuracy, 'g.-', label='Train')
plt.plot(time, test_accuracy, 'r.-', label='Test')
plt.legend()

Time to achieve a given accuracy

#plot
plt.title('CIFAR-10 time by accuracy')
plt.xlabel('Accuracy')
plt.ylabel('Time (min)')
plt.grid(True)
plt.plot(train_accuracy, time, 'g.-', label='Train')
plt.plot(test_accuracy, time, 'r.-', label='Test')
plt.legend()

# Multi Input

In [5]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, concatenate

In [6]:
# feature extraction from gray scale image
inputs = Input(shape = (28,28,1))
 
conv1 = Conv2D(16, (3,3), activation = 'relu', padding = "SAME")(inputs)
pool1 = MaxPooling2D(pool_size = (2,2), strides = 2)(conv1)
conv2 = Conv2D(32, (3,3), activation = 'relu', padding = "SAME")(pool1)
pool2 = MaxPooling2D(pool_size = (2,2), strides = 2)(conv2)
flat_1 = Flatten()(pool2)
 
# feature extraction from RGB image
inputs_2 = Input(shape = (28,28,3))
 
conv1_2 = Conv2D(16, (3,3), activation = 'relu', padding = "SAME")(inputs_2)
pool1_2 = MaxPooling2D(pool_size = (2,2), strides = 2)(conv1_2)
conv2_2 = Conv2D(32, (3,3), activation = 'relu', padding = "SAME")(pool1_2)
pool2_2 = MaxPooling2D(pool_size = (2,2), strides = 2)(conv2_2)
flat_2 = Flatten()(pool2_2)
 
# concatenate both feature layers and define output layer after some dense layers
concat = concatenate([flat_1,flat_2])
dense1 = Dense(512, activation = 'relu')(concat)
dense2 = Dense(128, activation = 'relu')(dense1)
dense3 = Dense(32, activation = 'relu')(dense2)
output = Dense(10, activation = 'softmax')(dense3)
 
# create model with two inputs
model = Model([inputs,inputs_2], output)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 28, 28, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 16)   160         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 28, 28, 16)   448         input_2[0][0]                    
______________________________________________________________________________________________

In [8]:
json_model = model.to_json()

In [9]:
%%sql
DROP TABLE IF EXISTS model_arch_library;
SELECT madlib.load_keras_model('model_arch_library', :json_model ::json,
                               NULL,                  -- Weights
                               'Multi Output MNIST',              -- Name
                               'Input 1: greyscale, Input 2: color' -- Descr
);

SELECT * FROM model_arch_library;

 * postgresql://localhost:6000/mnist
Done.
1 rows affected.
1 rows affected.


[(1, {u'class_name': u'Model', u'keras_version': u'2.2.4-tf', u'config': {u'layers': [{u'class_name': u'InputLayer', u'inbound_nodes': [], u'config': {u'dt ... (7129 characters truncated) ... '}], u'input_layers': [[u'input_1', 0, 0], [u'input_2', 0, 0]], u'name': u'model', u'output_layers': [[u'dense_3', 0, 0]]}, u'backend': u'tensorflow'}, None, u'Multi Output MNIST', u'Input 1: greyscale, Input 2: color', u'__madlib_temp_31588424_1615918903_7880856__')]

## Create color mnist input from greyscale input

In [ ]:
from tensorflow.keras.datasets import mnist
import numpy as np
import PIL

In [ ]:
%%sql
DROP TABLE IF EXISTS random_colors_train;
CREATE TABLE random_colors_train AS
    SELECT id,
        floor(r/(r+b+g)*256)::INT AS r,
        floor(g/(r+b+g)*256)::INT AS g,
        floor(b/(r+b+g)*256)::INT AS b FROM
            (SELECT s.id,random() as r, random() as b, random() as g FROM
                (SELECT generate_series(1,60000) as id) s) x;

In [ ]:
%%sql
DROP TABLE IF EXISTS random_colors_test;
CREATE TABLE random_colors_test AS
    SELECT id,
        floor(r/(r+b+g)*256)::INT AS r,
        floor(g/(r+b+g)*256)::INT AS g,
        floor(b/(r+b+g)*256)::INT AS b FROM
            (SELECT s.id,random() as r, random() as b, random() as g FROM
                (SELECT generate_series(1,10000) as id) s) x;

In [12]:
%sql SET search_path=public,madlib

 * postgresql://localhost:6000/mnist
Done.


[]

In [ ]:
%%sql
DROP TABLE IF EXISTS mnist_color_train;
CREATE TABLE mnist_color_train AS
    SELECT id,
        ARRAY[
            array_scalar_mult(x,(r/256.0)::REAL),
            array_scalar_mult(x,(g/256.0)::REAL),
            array_scalar_mult(x,(b/256.0)::REAL)
        ] AS x,
        y
    FROM mnist_train_data JOIN random_colors_train USING (id);

In [ ]:
%%sql
DROP TABLE IF EXISTS mnist_color_test;
CREATE TABLE mnist_color_test AS
    SELECT id,
        ARRAY[
            array_scalar_mult(x,(r/256.0)::REAL),
            array_scalar_mult(x,(g/256.0)::REAL),
            array_scalar_mult(x,(b/256.0)::REAL)
        ] AS x,
        y
    FROM mnist_test_data JOIN random_colors_test USING (id);

In [ ]:
%%sql
DROP TABLE IF EXISTS mnist_grey_color_train;
CREATE TABLE mnist_grey_color_train AS
    SELECT id, ARRAY[g.x] AS x_grey, c.x AS x_color, y
FROM mnist_train_data g JOIN mnist_color_train c USING (id,y);

In [ ]:
%%sql
DROP TABLE IF EXISTS mnist_grey_color_test;
CREATE TABLE mnist_grey_color_test AS
    SELECT id, ARRAY[g.x] AS x_grey, c.x AS x_color, y
FROM mnist_test_data g JOIN mnist_color_test c USING (id,y);

## Convert data format from channels first to channels last

In [ ]:
import pandas as pd
import numpy as np
import PIL

In [ ]:
gcd_test = %sql SELECT * FROM mnist_grey_color_test ORDER BY id

In [ ]:
gcd_train = %sql SELECT * FROM mnist_grey_color_train ORDER BY id

In [ ]:
gcd_test = gcd_test.dict()
gcd_train = gcd_train.dict()

In [ ]:
x_grey = [ np.asarray(x) for x in gcd_test['x_grey']]
x_color = [ np.asarray(x) for x in gcd_test['x_color'] ]

In [ ]:
grey_color_test_dataset = { 'id' : gcd_test['id'], 'x_grey' : x_grey, 'x_color' : x_color, 'y' : gcd_test['y'] }

In [ ]:
x_grey = [ np.asarray(x) for x in gcd_train['x_grey'] ]
x_color = [ np.asarray(x) for x in gcd_train['x_color'] ]

In [ ]:
grey_color_train_dataset = { 'id' : gcd_train['id'], 'x_grey' : x_grey, 'x_color' : x_color, 'y' : gcd_train['y'] }

In [ ]:
df_test = pd.DataFrame(grey_color_test_dataset)
df_test.set_index('id')

In [ ]:
df_train = pd.DataFrame(grey_color_train_dataset)
df_train.set_index('id')

In [ ]:
df_train.loc[1]['x_color'].shape, df_train.loc[1]['x_grey'].shape

In [ ]:
def channels_first_to_last(row):
    row['x_grey'] = np.transpose(row['x_grey'], (1,2,0))
    row['x_color'] = np.transpose(row['x_color'], (1,2,0))
    return row

In [ ]:
df_test = df_test.apply( channels_first_to_last, axis=1 )

In [ ]:
df_train = df_train.apply( channels_first_to_last, axis=1 )

In [ ]:
from sqlalchemy import create_engine, INTEGER, ARRAY, REAL, SMALLINT

In [ ]:
engine = create_engine('postgres://localhost:6000/mnist')

In [ ]:
res = engine.execute("SELECT version()")

In [ ]:
res.fetchall()

In [ ]:
df_train.loc[1]['x_color'].shape, df_train.loc[1]['x_grey'].shape

In [ ]:
df = df_test

In [ ]:
df['x_color'] = df['x_color'].apply(lambda x : x.tolist())
df['x_grey'] = df['x_grey'].apply(lambda x : x.tolist())

In [ ]:
df_test = df

In [ ]:
df = df_train

In [ ]:
df['x_color'] = df['x_color'].apply(lambda x : x.tolist())
df['x_grey'] = df['x_grey'].apply(lambda x : x.tolist())

In [ ]:
df_train = df

In [ ]:
dtype = { 'id' : INTEGER, 'x_grey' : ARRAY(REAL), 'x_color' : ARRAY(REAL), 'y' : SMALLINT }

In [ ]:
df_test.to_sql('channels_last_test', con=engine, if_exists='replace', dtype=dtype, chunksize=500, method='multi', schema='public', index=False)

In [ ]:
df_train.to_sql('channels_last_train', con=engine, if_exists='replace', dtype=dtype, chunksize=1000, method='multi', schema='public', index=False)

In [ ]:
grey1 = np.array(df_train['x_grey'][12], dtype='uint8')

In [ ]:
color1 = np.array(df_train['x_color'][12], dtype='uint8')

In [ ]:
grey1.shape, color1.shape

In [ ]:
grey1slim = grey1.reshape((28,28))

In [ ]:
grey1slim.shape

In [ ]:
img1g = PIL.Image.fromarray(grey1slim)

In [ ]:
img1c = PIL.Image.fromarray(color1)

In [ ]:
img1g.show() # Doesn't work for me

In [ ]:
img1g.save('img1g.jpg')
img1c.save('img1c.jpg')

In [ ]:
!display img1g.jpg

In [ ]:
!display img1c.jpg

In [ ]:
%sql SELECT id, array_dims(x_grey), array_dims(x_color), y FROM channels_last_test ORDER BY id LIMIT 5

In [ ]:
%%sql
-- Careful not to drop unless channels_last_test exists, otherwise we lose the table and will have to re-create
--drop table if exists mnist_grey_color_test;
alter table channels_last_test rename to mnist_grey_color_test;

In [ ]:
%sql SELECT id, array_dims(x_grey), array_dims(x_color), y FROM channels_last_train ORDER BY id LIMIT 5

In [ ]:
%%sql
-- Careful not to drop unless channels_last_train exists, otherwise we lose the table and will have to re-create
--drop table if exists mnist_grey_color_train;
alter table channels_last_train rename to mnist_grey_color_train;

## Minbatch input tables

In [ ]:
%%sql
DROP TABLE IF EXISTS mnist_color_train_packed, mnist_color_train_packed_summary;
SELECT madlib.training_preprocessor_dl('mnist_grey_color_train',        -- Source table
                                       'mnist_grey_color_train_packed', -- Output table
                                       'y',                          -- Dependent variable
                                       'x_grey, x_color',            -- Independent variables
                                        1000,                        -- Buffer size
                                        255                          -- Normalizing constant
                                        );

SELECT * FROM mnist_train_data_packed_summary;

In [ ]:
%%sql
DROP TABLE IF EXISTS mnist_test_data_packed, mnist_test_data_packed_summary;
SELECT madlib.training_preprocessor_dl('mnist_grey_color_test',        -- Source table
                                       'mnist_grey_color_test_packed', -- Output table
                                       'y',                          -- Dependent variable
                                       'x_grey,x_color',             -- Independent variables
                                        1000,                        -- Buffer size
                                        255                          -- Normalizing constant
                                        );

SELECT * FROM mnist_train_data_packed_summary;

## Run fit on multi-input dataset

In [11]:
%%sql
DROP TABLE IF EXISTS mnist_multout_model, mnist_multout_model_summary;
SELECT madlib.madlib_keras_fit('mnist_grey_color_train_packed',    -- source table
                               'mnist_multiin_model',           -- model output table
                               'model_arch_library',            -- model arch table
                                1,                              -- model arch id
                                $$ loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']$$,  -- compile_params
                                $$ batch_size=256, epochs=1, shuffle=True $$,  -- fit_params
                                3,                              -- num_iterations
                                FALSE,                          -- use GPUs
                                'mnist_grey_color_test_packed',    -- validation dataset
                                2                               -- metrics compute frequency
                              );

 * postgresql://localhost:6000/mnist
Done.
1 rows affected.


madlib_keras_fit
""
